# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests as req
import xlsxwriter 
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
response = req.get(api_url)
print(response.status_code)
data = response.json()
print(data)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_54614b712a594029be959c2531524c48
200
{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '(NANS/DMERB) TSEQLGACLKEST OLG AA', 'calculationPrice': 'close', 'open': 126.65, 'openTime': 1686105463138, 'openSource': 'alfiofic', 'close': 128.5, 'closeTime': 1694509764667, 'closeSource': 'afilfoic', 'high': 128.8, 'highTime': 1671025451202, 'highSource': 'ecme y5aed pl1i nudtier', 'low': 124.83, 'lowTime': 1682374583511, 'lowSource': 'nipmeautde5cldr1y  ie e', 'latestPrice': 128.89, 'latestSource': 'Close', 'latestTime': 'May 28, 2021', 'latestUpdate': 1699157546766, 'latestVolume': 74460981, 'iexRealtimePrice': 127.69, 'iexRealtimeSize': 2666, 'iexLastUpdated': 1646871745620, 'delayedPrice': 128.16, 'delayedPriceTime': 1641928583171, 'oddLotDelayedPrice': 130.03, 'oddLotDelayedPriceTime': 1666973953851, 'extendedPrice': 130.82, 'extendedChange': 0.01, 'extendedChangePercent': 8e-05, 'extendedPriceTime': 16414900892

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns =['Ticker', 'Stock Price', 'Market Capitalization',' Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

In [7]:
series = pd.Series(
 [
     symbol,
     price,
     market_cap,
     'N/A'
 ],
index = my_columns)

In [29]:
final_dataframe = final_dataframe.append(series,ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,142.28,44030707348,N/A
1,AAL,25.34,15548238295,N/A
2,AAP,190.34,12862687326,N/A
3,AAPL,126.78,2142169643190,N/A
4,ABBV,116.90,205484999896,N/A
...,...,...,...,...
601,ZBH,169.45,36203280253,N/A
602,ZBRA,515.43,27227587741,N/A
603,ZION,59.54,9733843492,N/A
604,ZTS,183.89,84154965691,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [15]:
'''
print('Getting all stocks')
for ticker in stocks['Ticker']:
    print(f'getting data for {ticker}')
    api_url = f'https://sandbox.iexapis.com/stable/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = req.get(api_url).json()
    price = data['latestPrice']
    market_cap = data['marketCap']
    series = pd.Series([ticker, price, market_cap, 'N/A'],index = my_columns)
    final_dataframe = final_dataframe.append(series, ignore_index = True)
print('Done Getting all stocks')
final_dataframe
'''

<Response [400]>

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [16]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [50]:

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for symbol_group in symbol_groups:
    symbol_strings.append(','.join(symbol_group))

final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = req.get(batch_api_call_url).json();
    for symbol in symbol_string.split(','):
        quote = data[symbol]['quote']
        price = quote['latestPrice']
        market_cap = quote['marketCap']
        series = pd.Series([symbol, price, market_cap, 'N/A'],index = my_columns)
        final_dataframe = final_dataframe.append(series, ignore_index = True)

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,141.20,42503664708,N/A
1,AAL,25.13,15883719455,N/A
2,AAP,189.75,12817238919,N/A
3,AAPL,125.73,2147170944172,N/A
4,ABBV,115.50,203132750988,N/A
...,...,...,...,...
500,YUM,121.71,36042789353,N/A
501,ZBH,170.08,35206842383,N/A
502,ZBRA,506.26,27337614719,N/A
503,ZION,59.79,9801677104,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [51]:
portfolio_size = 10000 # input('Ener the value of your portfolio')
for i in range(0, len(final_dataframe.index)):
    qty = math.floor((portfolio_size/500)/final_dataframe.loc[i, 'Stock Price'])
    final_dataframe.loc[i, 'ToBuy'] = qty
    

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,ToBuy
0,A,141.20,42503664708,N/A,0.0
1,AAL,25.13,15883719455,N/A,0.0
2,AAP,189.75,12817238919,N/A,0.0
3,AAPL,125.73,2147170944172,N/A,0.0
4,ABBV,115.50,203132750988,N/A,0.0
...,...,...,...,...,...
500,YUM,121.71,36042789353,N/A,0.0
501,ZBH,170.08,35206842383,N/A,0.0
502,ZBRA,506.26,27337614719,N/A,0.0
503,ZION,59.79,9801677104,N/A,0.0


In [54]:
final_dataframe.query('ToBuy > 0')

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,ToBuy
19,AIV,7.390,1068119703,N/A,2.0
30,AMCR,11.800,19151240263,N/A,1.0
107,COG,16.500,6711754808,N/A,1.0
111,COTY,9.070,7122843044,N/A,2.0
118,CTL,11.000,12548716274,N/A,1.0
174,F,14.660,59734663638,N/A,1.0
194,FTI,9.000,3908165301,N/A,2.0
198,GE,14.590,124909640999,N/A,1.0
214,HBAN,16.260,16629023925,N/A,1.0
225,HPE,16.490,21522930039,N/A,1.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [49]:
writer = pd.ExcelWriter("recommendedTrades.xlsx", engine = 'xlsxwriter')
final_dataframe.to_excel(writer, "Sheet1", index=False)
writer.save()

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: